Задние №1
С использованием системных вызовов fork, execve разработать программу на языке ассемблер,
в которой родительский процесс ожидает ввода команды, в качестве которой могут подаваться названия
исполняемых файлов пятой и шестой лабораторных работ. После получения команды исполняемый файл запускается,
а родительский процесс переходит в режим ожидания следующей команды.

Задание №2
Разработать программу на языке ассемблера, в которой область памяти заполняется заданным количеством случайных чисел.
В соответствии с вариантом запустить несколько процессов, каждый из которых выполняет заданный набор действий над
полученным масссивом случайных чисел. Каждый процесс должен вывести свои результаты на экран, после чего завершить работу.
Родительский процесс должен завершить работу как только отработают все дочерние процессы.

[794,
 ['Треды (clone), куча'],
 ['Пятое после миниманого',
  'Третье после максимального',
  'Количество чисел кратных пяти',
  'Среднее арифметическое значение (округленное до целого)']]

# ЗАДАЧА №1

```ASM
format ELF64
public _start
public exit

section '.data' writeable
    cmd_buffer      rb 256
    argv_ptrs       rq 10

    path_lab5       db './Asm10', 0
    path_lab6       db './Asm6', 0

    str_lab5        db 'Asm10', 0
    str_lab6        db 'Asm6', 0
    str_q           db 'q', 0

    child_pid       dq 0
    wait_status     dq 0

    envp            dq 0

    prompt          db '> ', 0
    msg_unknown     db 'Unknown command', 10, 0
    msg_error_fork  db 'Fork failed', 10, 0

section '.text' executable
_start:
    lea rbx, [rsp + 8]      ; rbx = адрес первого элемента argv (пропускаем argc)

    .find_envp_loop:
        cmp qword [rbx], 0      ; Проверяем, не дошли ли мы до NULL (конец argv)
        je .found_null
        add rbx, 8              ; Переходим к следующему элементу
        jmp .find_envp_loop

    .found_null:
        add rbx, 8              ; Пропускаем сам NULL. Теперь rbx указывает на начало envp
        mov [envp], rbx         ; Сохраняем правильный указатель!

    .main_loop:
        ; Вывод приглашения
        mov rax, 1
        mov rdi, 1
        mov rsi, prompt
        mov rdx, 2
        syscall

        ; Чтение ввода
        mov rax, 0
        mov rdi, 0
        mov rsi, cmd_buffer
        mov rdx, 255
        syscall

        cmp rax, 0
        jle exit

        ; Удаление \n
        mov rcx, rax
        dec rcx
        cmp byte [cmd_buffer + rcx], 10
        jne .parse_input
        mov byte [cmd_buffer + rcx], 0

    .parse_input:
        mov rsi, cmd_buffer
        mov rdi, argv_ptrs
        xor rcx, rcx

    .token_loop:
        cmp byte [rsi], ' '
        je .skip_space
        cmp byte [rsi], 0
        je .tokens_done

        mov [rdi + rcx*8], rsi
        inc rcx

    .scan_word:
        inc rsi
        cmp byte [rsi], ' '
        je .end_word
        cmp byte [rsi], 0
        je .tokens_done
        jmp .scan_word

    .end_word:
        mov byte [rsi], 0
        inc rsi
        jmp .token_loop

    .skip_space:
        inc rsi
        jmp .token_loop

    .tokens_done:
        mov qword [rdi + rcx*8], 0

        cmp rcx, 0
        je .main_loop

        ; --- Сравнение ---
        mov rsi, [argv_ptrs]
        mov rdi, str_q
        call strcmp
        test rax, rax
        jz exit

        mov rsi, [argv_ptrs]
        mov rdi, str_lab5
        call strcmp
        test rax, rax
        jz .run_lab5

        mov rsi, [argv_ptrs]
        mov rdi, str_lab6
        call strcmp
        test rax, rax
        jz .run_lab6

        mov rax, 1
        mov rdi, 1
        mov rsi, msg_unknown
        mov rdx, 16
        syscall
        jmp .main_loop

    .run_lab5:
        mov rbx, path_lab5
        jmp .do_fork

    .run_lab6:
        mov rbx, path_lab6
        jmp .do_fork

    .do_fork:
        mov rax, 57 ; создаём копию основного процесса
        syscall

        cmp rax, 0
        jl .fork_error
        je .child_process

        mov [child_pid], rax
        jmp .wait_child

    .child_process:
        mov rax, 59             ; sys_execve
        mov rdi, rbx            ; Имя файла
        mov rsi, argv_ptrs      ; Аргументы
        mov rdx, [envp]         ; Передаем найденное окружение
        syscall

        call exit_error

    .wait_child:
        mov rax, 61 ; sys_wait4
        mov rdi, [child_pid]
        mov rsi, wait_status
        mov rdx, 0
        mov r10, 0
        syscall
        jmp .main_loop

    .fork_error:
        mov rax, 1
        mov rdi, 1
        mov rsi, msg_error_fork
        mov rdx, 12
        syscall
        jmp .main_loop

exit_error:
    mov rax, 60
    mov rdi, 1
    syscall

exit:
    mov rax, 60
    xor rdi, rdi
    syscall

strcmp:
    push rsi
    push rdi
    push rbx

    .loop:
        mov al, [rsi]
        mov bl, [rdi]
        cmp al, bl
        jne .not_equal
        test al, al
        jz .equal
        inc rsi
        inc rdi
        jmp .loop
    .not_equal:
        mov rax, 1
        jmp .done
    .equal:
        xor rax, rax
    .done:
        pop rbx
        pop rdi
        pop rsi
        ret
```

# Задача №2
```ASM
format ELF64
public _start
ARRAY_LEN       = 10
TOTAL_THREADS   = 4

section '.data' writeable
    msg_gen         db '[Parent] Генерируем 754 рандомных числа', 10, 0
    msg_arr         db '[Parent] Элементы массива: ', 0    ; <-- Добавлено для вывода
    msg_space       db ' ', 0                              ; <-- Добавлено (разделитель)
    msg_task1       db '[Task 1] 5-ое число после минимального (значение: ', 0
    msg_task1_idx   db ', по индексу: ', 0
    msg_res         db ') -> Результат: ', 0
    msg_task2       db '[Task 2] 3-ий число после максимального (значение: ', 0
    msg_task2_idx   db ', по индексу: ', 0
    msg_task3       db '[Task 3] Количество чисел кратных 5: ', 0
    msg_task4       db '[Task 4] Среднее значение (округление до целого): ', 0
    newline         db 10, 0
    fmt_num_buf     rb 20
    array_ptr       dq 0
    seed            dq 123456789

section '.text' executable
_start:
    mov rax, 9                  ; sys_mmap
    mov rdi, 0
    mov rsi, 8192               ; Размер памяти

    mov rdx, 3                  ; 1 (READ) + 2 (WRITE) = 3

    mov r10, 34                 ; 2 (PRIVATE) + 32 (ANONYMOUS, 0x20) = 34 (0x22)

    mov r8, -1
    mov r9, 0
    syscall

    cmp rax, 0
    jl exit_error
    mov [array_ptr], rax

    ; --- 2. ЗАПОЛНЕНИЕ МАССИВА ---
    mov rdi, [array_ptr]
    mov rcx, ARRAY_LEN

    .fill_loop:
        call rand
        mov [rdi], rax
        add rdi, 8
        loop .fill_loop

        mov rsi, msg_gen
        call print_string

        ; --- ВСТАВКА: ВЫВОД МАССИВА ---
        mov rsi, msg_arr        ; Выводим заголовок "Элементы массива: "
        call print_string

        mov rbx, [array_ptr]    ; Указатель на начало массива
        mov rcx, ARRAY_LEN      ; Счётчик цикла
        xor r12, r12            ; Индекс (используем r12, т.к. он callee-saved и не портится функциями print)

    .print_loop:
        mov rax, [rbx + r12*8]  ; Загружаем число из массива
        call print_uint         ; Выводим число

        mov rsi, msg_space      ; Выводим пробел
        call print_string

        inc r12
        loop .print_loop        ; rcx уменьшается автоматически

        call print_newline      ; Перенос строки после вывода массива
        ; ------------------------------

        ; --- 3. ЗАПУСК ПРОЦЕССОВ (FORK) ---

        call fork_process
        test rax, rax
        jz task_min_plus_5

        call fork_process
        test rax, rax
        jz task_max_plus_3

        call fork_process
        test rax, rax
        jz task_count_div_5

        call fork_process
        test rax, rax
        jz task_average

        ; --- 4. ОЖИДАНИЕ ЗАВЕРШЕНИЯ ---
        mov rcx, TOTAL_THREADS
    .wait_loop:
        push rcx
        mov rax, 61
        mov rdi, -1
        mov rsi, 0
        mov rdx, 0
        mov r10, 0
        syscall
        pop rcx
        loop .wait_loop

        call exit

task_min_plus_5:
    mov rbx, [array_ptr]
    mov rcx, ARRAY_LEN
    mov rax, [rbx]              ; Мин. значение
    xor rdx, rdx                ; Индекс мин.
    xor r8, r8

    .loop_min:
        cmp [rbx + r8*8], rax
        jge .skip_min
        mov rax, [rbx + r8*8]
        mov rdx, r8
    .skip_min:
        inc r8
        cmp r8, rcx
        jl .loop_min

        push rax
        push rdx
        mov rsi, msg_task1
        call print_string
        pop rdx
        push rdx
        mov rax, [rsp+8]
        call print_uint
        mov rsi, msg_task1_idx
        call print_string
        pop rdx
        mov rax, rdx
        call print_uint
        mov rsi, msg_res
        call print_string

        add rdx, 5
        mov rax, rdx
        xor rdx, rdx
        mov r9, ARRAY_LEN
        div r9

        mov rbx, [array_ptr]
        mov rax, [rbx + rdx*8]
        call print_uint
        call print_newline
        call exit_thread

task_max_plus_3:
    mov rbx, [array_ptr]
    mov rcx, ARRAY_LEN
    mov rax, [rbx]
    xor rdx, rdx
    xor r8, r8

    .loop_max:
        cmp [rbx + r8*8], rax
        jle .skip_max
        mov rax, [rbx + r8*8]
        mov rdx, r8
    .skip_max:
        inc r8
        cmp r8, rcx
        jl .loop_max

        push rax
        push rdx
        mov rsi, msg_task2
        call print_string
        mov rax, [rsp+8]
        call print_uint
        mov rsi, msg_task2_idx
        call print_string
        pop rdx
        mov rax, rdx
        call print_uint
        mov rsi, msg_res
        call print_string

        add rdx, 3
        mov rax, rdx
        xor rdx, rdx
        mov r9, ARRAY_LEN
        div r9

        mov rbx, [array_ptr]
        mov rax, [rbx + rdx*8]
        call print_uint
        call print_newline
        call exit_thread

task_count_div_5:
    mov rbx, [array_ptr]
    mov rcx, ARRAY_LEN
    xor r12, r12
    xor r8, r8
    mov r9, 5

    .loop_div:
        mov rax, [rbx + r8*8]
        xor rdx, rdx
        div r9
        cmp rdx, 0
        jne .next_div
        inc r12
    .next_div:
        inc r8
        cmp r8, rcx
        jl .loop_div

        mov rsi, msg_task3
        call print_string
        mov rax, r12
        call print_uint
        call print_newline
        call exit_thread

task_average:
    mov rbx, [array_ptr]
    mov rcx, ARRAY_LEN
    xor r12, r12
    xor r8, r8

    .loop_avg:
        add r12, [rbx + r8*8]
        inc r8
        cmp r8, rcx
        jl .loop_avg

        mov rax, r12
        xor rdx, rdx
        mov r9, ARRAY_LEN
        div r9

        mov rsi, msg_task4
        call print_string
        call print_uint
        call print_newline
        call exit_thread

; Используем FORK (57) вместо CLONE.
; Fork создает точную копию процесса.
; В родителе возвращает PID ребенка, в ребенке возвращает 0.
fork_process:
    mov rax, 57
    syscall
    ret

rand:
    ; ВАЖНО: Сохраняем RCX, так как он используется как счетчик цикла снаружи!
    push rcx

    mov rax, [seed]
    mov rbx, 6364136223846793005
    mul rbx

    mov rcx, 1442695040888963407
    add rax, rcx

    mov [seed], rax
    xor rdx, rdx
    mov rbx, 1000
    div rbx
    mov rax, rdx

    pop rcx
    ret

print_string:
    push rdi
    push rax
    push rdx
    push rcx
    mov rdi, rsi
    call strlen
    mov rdx, rax
    mov rax, 1
    mov rdi, 1
    syscall
    pop rcx
    pop rdx
    pop rax
    pop rdi
    ret

print_newline:
    mov rsi, newline
    call print_string
    ret

print_uint:
    push rbx
    push rcx
    push rdx
    push rsi
    push rdi
    mov rbx, 10
    mov rcx, fmt_num_buf
    add rcx, 19
    mov byte [rcx], 0

    .convert_loop:
        dec rcx
        xor rdx, rdx
        div rbx
        add dl, '0'
        mov [rcx], dl
        test rax, rax
        jnz .convert_loop
        mov rsi, rcx
        call print_string
        pop rdi
        pop rsi
        pop rdx
        pop rcx
        pop rbx
        ret

strlen:
    xor rax, rax
    .loop:
        cmp byte [rdi + rax], 0
        je .done
        inc rax
        jmp .loop
    .done:
        ret

exit_thread:
    mov rax, 60
    xor rdi, rdi
    syscall

exit_error:
    mov rax, 60
    mov rdi, 1
    syscall

exit:
    mov rax, 60
    xor rdi, rdi
    syscall
```